
# **😎 Age and Gender Estimation in TensorFlow ( Workbook 1, Age Estimation )**

* For Gender classification, go to [Workbook 2 ( Gender Classification )](https://colab.research.google.com/drive/1aOMbQYK2bvedNVXkZvHC_FsBsKrCx01P?usp=sharing)

In this notebook, we train a Keras model to estimate the age of a person, given a *face-cropped* image. We use the famous [UTKFace Dataset](https://susanqq.github.io/UTKFace/), which contains 23K images where each image is labelled with its gender, age and ethinicity.


> **Note: Please make sure that you are connected to the GPU runtime of Google Colab. Else, the training might take a decade long. Go to Runtime > Change runtime type > Hardware accelerator.**



## 1) **Downloading the UTKFace dataset** 💻


The UTKFace dataset is available as a Google Drive folder. See the file [here](https://drive.google.com/drive/folders/0BxYys69jI14kU0I1YUQyY1ZDRUE). Download it place it in your Google Drive.

Once completed, mount the Google Drive in this notebook.


In [ ]:

from google.colab import drive

drive.mount('/content/drive')



Unzip the `utkface_23k.zip` file to the local directory. This would significantly increase the pace at which images are parsed. See [this](https://stackoverflow.com/a/60802388/10878733) answer on StackOverflow.


In [ ]:

# Replace with your path!
!unzip -q /content/drive/MyDrive/Datasets/utkface_23k.zip -d data



Rename the directory from `utkace_23k` to `utkface23k`, in order to remove the underscore, which might cause errors while we're splitting the filename, to get the age of the person.


In [ ]:

!mv data/utkface_23k data/utkface23k


In [ ]:

# To download checkpoints, Keras models, TFLite models
from google.colab import files

# Life is incomplete without this statement!
import tensorflow as tf

# And this as well!
import numpy as np

# To visualize results
import matplotlib.pyplot as plt

import os
import datetime



## 2) **Processing the data** 🦾

Once we've downloaded the dataset, we need to perform the following operations on the dataset, so that it can be used for training our model,

* 👉🏻 Reading the image files as 3D NumPy arrays. Note, we'll use 3-channeled RGB images for training the model, so each array will have a shape of `[ img_width , img_height , 3 ]`.

* 👉🏻 Split the filename so as to parse the age of the person in corresponding image. We use the `tf.strings.split()` method for performing this task.

* 👉🏻 The maximum value of our target variable is $116$ years which is used for normalizing the `age` variable.


Once this operations have been performed, we are left with $N$ samples where each sample consists of image array `[ 200 , 200 , 3 ]` and its corresponding label, the age of that person, which has a shape `[ 1 , ]`

We'll use `tf.data.Dataset` as it helps us to process the data faster, taking advantage of parallel computing. The above two operations will be mapped on each filename using `tf.data.Dataset.map` method.


In [ ]:

# Image size for our model.
MODEL_INPUT_IMAGE_SIZE = [ 200 , 200 ]

# Fraction of the dataset to be used for testing.
TRAIN_TEST_SPLIT = 0.3

# Number of samples to take from dataset
N = 20000

# This method will be mapped for each filename in `list_ds`. 
def parse_image( filename ):

    # Read the image from the filename and resize it.
    image_raw = tf.io.read_file( filename )
    image = tf.image.decode_jpeg( image_raw , channels=3 ) 
    image = tf.image.resize( image , MODEL_INPUT_IMAGE_SIZE ) / 255

    # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split( tf.strings.split( filename , '/' )[ 2 ] , '_' )

    # Normalize
    age = tf.strings.to_number( parts[ 0 ] ) / 116

    return image , age

# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files( 'data/utkface23k/*' , shuffle=True )

# Map `parse_image` method to all filenames.
dataset = list_ds.map( parse_image , num_parallel_calls=tf.data.AUTOTUNE )
dataset = dataset.take( N )



We create two splits from our dataset, one for training the model and another for testing the model. The fraction of the dataset which will be used for testing the model is determined by `TRAIN_TEST_SPLIT`.



In [ ]:

# Create train and test splits of the dataset.
num_examples_in_test_ds = int( dataset.cardinality().numpy() * TRAIN_TEST_SPLIT )

test_ds = dataset.take( num_examples_in_test_ds )
train_ds = dataset.skip( num_examples_in_test_ds )

print( 'Num examples in train ds {}'.format( train_ds.cardinality() ) )
print( 'Num examples in test ds {}'.format( test_ds.cardinality() ) )



## 3) **The CNN Model** 👨‍🎓

In our approach, we treat age estimation as a regression problem. Our aim is to develop a model which has lesser parameters ( which implies lesser inference time and size ) but powerful enough so that it can generalize better.

* 👉🏻 The model takes in a batch of shape `[ None , 200 , 200 , 3 ]` and performs a number of convolutions on it as determined by `num_blocks`.
* 👉🏻 Each block consists of a sequence of layers : `Conv2D -> BatchNorm -> LeakyReLU`

```
# Define the conv block.
if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides 
                                            , use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
```

* If `lite_model` is set to `True`, we use [Separable Convolutions](https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728) which have lesser parameters. We could achieve a *faster* model, compromising its performance.

* 👉🏻 We stack such`num_blocks` blocks sequentially, where the no. of filters for each layer is taken from `num_filters`.

* 👉🏻 Next we add a number of `Dense` layers to learn the features extracted by convolutional layers. Note, we also add a `Dropout` layer, to reduce overfitting. The `rate` for each `Dropout` layer is decreased subsequently for each layer, so that the learnability of `Dense` layer with lesser units ( neurons ) is not affected.

```
def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x
```

> See [this](https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/) blog for choosing the weight decay values used in the above two blocks.

* 👉🏻 The output of the model is a tensor with shape `[ None, 1 ]`


In [ ]:

# Negative slope coefficient for LeakyReLU.
leaky_relu_alpha = 0.2

lite_model = False

# Define the conv block.
def conv( x , num_filters , kernel_size=( 3 , 3 ) , strides=1 ):
    if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides, 
                                            use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
    return x

def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x


# No. of convolution layers to be added.
num_blocks = 6
# Num filters for each conv layer.
num_filters = [ 16 , 32 , 64 , 128 , 256 , 256 ]
# Kernel sizes for each conv layer.
kernel_sizes = [ 3 , 3 , 3 , 3 , 3 , 3 ]

# Init a Input Layer.
inputs = tf.keras.layers.Input( shape=MODEL_INPUT_IMAGE_SIZE + [ 3 ] )

# Add conv blocks sequentially
x = inputs
for i in range( num_blocks ):
    x = conv( x , num_filters=num_filters[ i ] , kernel_size=kernel_sizes[ i ] )
    x = tf.keras.layers.MaxPooling2D()( x )

# Flatten the output of the last Conv layer.
x = tf.keras.layers.Flatten()( x )
conv_output = x 

# Add Dense layers ( Dense -> LeakyReLU -> Dropout )
x = dense( conv_output , 256 , 0.6 )
x = dense( x , 64 , 0.4 )
x = dense( x , 32 , 0.2 )
outputs = tf.keras.layers.Dense( 1 , activation='relu' )( x )

# Build the Model
model = tf.keras.models.Model( inputs , outputs )

# Uncomment the below to view the summary of the model.
model.summary()
# tf.keras.utils.plot_model( model , to_file='architecture.png' )



## 4) **Compiling the model ( and other callbacks )** 🧱

Once we've defined the architecture for our model, we'll compile our Keras model and also initialize some useful callbacks.

* 👉🏻 As we're performing regression, we'll use the Mean Absolute Error ( MAE ) loss function. See [`tf.keras.losses.mean_absolute_error`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MAE) for more details.

* 👉🏻 We'll use the Adam optimizer for training our model. See [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) for more details.

* 👉🏻 For evaluating the performance of our model, we use Mean Absolute Error as a metric. See [`tf.keras.metrics.MeanAbsoluteError`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/MeanAbsoluteError) for more details.


#### Callbacks:

* 👉🏻 [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to save the Keras model as an H5 file after every epoch.

* 👉🏻 [`tf.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) to visualize the training with TensorBoard.

* 👉🏻 [`tf.keras.callbacks.LearningRateScheduler`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LearningRateScheduler) to decrease the learning rate over a certain number of epochs, so as to make smaller steps, as the optimizer reaches near the minima of the loss function.

```
def scheduler( epochs , learning_rate ):
    if epochs < num_epochs * 0.25:
        return learning_rate
    elif epochs < num_epochs * 0.5:
        return 0.0005
    elif epochs < num_epochs * 0.75:
        return 0.0001
    else:
        return 0.000095
```

* 👉🏻 [`tf.keras.callbacks.EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to stop the training when the evaluation metric i.e the MAE stops improving on the test dataset.







In [ ]:

# Initial learning rate
learning_rate = 0.001

num_epochs = 50 #@param {type: "number"}
batch_size = 128 #@param {type: "number"}

# Batch and repeat `train_ds` and `test_ds`.
train_ds = train_ds.batch( batch_size )
test_ds = test_ds.batch( batch_size )

# Init ModelCheckpoint callback
save_dir_ = 'model_1'  #@param {type: "string"}
save_dir = save_dir_ + '/{epoch:02d}-{val_mae:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
    save_dir , 
    save_best_only=True , 
    monitor='val_mae' , 
    mode='min' , 
)

tb_log_name = 'model_1'  #@param {type: "string"}
# Init TensorBoard Callback
logdir = os.path.join( "tb_logs" , tb_log_name )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )

# Init LR Scheduler
def scheduler( epochs , learning_rate ):
    if epochs < num_epochs * 0.25:
        return learning_rate
    elif epochs < num_epochs * 0.5:
        return 0.0005
    elif epochs < num_epochs * 0.75:
        return 0.0001
    else:
        return 0.000095

lr_schedule_callback = tf.keras.callbacks.LearningRateScheduler( scheduler )

# Init Early Stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_mae' , patience=10 )

# Compile the model
model.compile( 
    loss=tf.keras.losses.mean_absolute_error ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'mae' ]
)



Run this cell to visualize the training of the model in TensorBoard ( in this notebook itself ).


In [ ]:

%load_ext tensorboard
%tensorboard --logdir tb_logs/



## 5) **Train and Evaluate the Model** 🏋🏻‍♂️

Start the training loop with all callbacks packed in.


In [ ]:

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds, 
    callbacks=[ checkpoint_callback , tensorboard_callback , lr_schedule_callback , early_stopping_callback ]
)


Evaluate the Model.

In [ ]:

p = model.evaluate( test_ds )
print( p )


( Optional ) Load a .h5 file for evaluation. Note: You need to run the code cell which creates `train_ds` and `test_ds`.

In [ ]:

batch_size = 128

model = tf.keras.models.load_model( 'model_lite_age.h5' )
p = model.evaluate( test_ds.batch( batch_size ) )
print( p )



Save the Keras model to the local disk, so that we can resume training if needed.


In [ ]:

model_name = 'model_age' #@param {type: "string"}
model_name_ = model_name + '.h5'

model.save( model_name_ )
files.download( model_name_ ) 



## 6) **Visualize the results**

We'll predict the age from some images taken from `test_ds` and plot them using `matplotlib`.




In [ ]:

fig = plt.figure( figsize=( 10 , 15 ) )
rows = 5
columns = 2

i = 1
for image , label in test_ds.unbatch().take( 10 ):
    image = image.numpy()
    fig.add_subplot( rows , columns , i )
    plt.imshow( image )
    label_ = int( model.predict( np.expand_dims( image , 0 ) ) * 116 )
    plt.axis( 'off' )
    plt.title( 'Predicted age : {} , actual age : {}'.format( label_ , int( label.numpy() * 116 ) ) )
    i += 1


 
## 7) **Convert to TensorFlow Lite format** 📡

Our model is to be deployed in an Android app, where we'll use [TF Lite Android](https://bintray.com/google/tensorflow/tensorflow-lite) package to parse the model and make predictions.

We use the `TFLiteConverter` API to convert our Keras Model ( `.h5` ) to a TF Lite buffer ( `.tflite` ). See the [official docs](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter/). We'll produce two TF Lite buffers, one with float16 quantization and other non-quantized model.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_types = [ tf.float16 ]
buffer = converter.convert()

open( '{}_q.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_q.tflite'.format( model_name ) )



For conversion to a non-quantized TF Lite buffer.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
buffer = converter.convert()

open( '{}_nonq.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_nonq.tflite'.format( model_name ) )



## Utility Methods

Use these methods to automate some of the tasks.


In [ ]:

#@title Utility to zip and download a directory
#@markdown Use this method to zip and download a directory. For ex. a TB logs 
#@markdown directory or a checkpoint(s) directory.

dir_to_zip = 'tb_logs' #@param {type: "string"}
output_filename = 'logs.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )


In [ ]:

#@title Utility to delete a directory
#@markdown Use this method to delete a directory. 

dir_path = ''  #@param {type: "string"}
os.system( f'rm -r {dir_path}')



# LICENSE

```
  
MIT License

Copyright (c) 2021 Shubham Panchal

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```
